In [1]:

import tensorflow as tf
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.activations import relu, softmax, sigmoid
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

**Mounting Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Function used**


*  To trim dataset:  https://drive.google.com/file/d/1QADrY4kxUQE5P3Nud32PFe5-a51zTZDu/view?usp=drive_link

compile(data_ptr -> pandas dataframe file, fill_mode -> str)

In [3]:
import pandas as pd
import seaborn as sns
data = pd.read_csv('/content/drive/My Drive/Statistella/train.csv')

In [48]:

data.drop([
"CODE_GENDER",
"DAYS_REGISTRATION",
"FLAG_EMP_PHONE",
"FLAG_MOBIL",
"FLAG_WORK_PHONE",
"HOUR_APPR_PROCESS_START",
"LIVE_CITY_NOT_WORK_CITY",
"LIVE_REGION_NOT_WORK_REGION",
"NAME_TYPE_SUITE",
"OCCUPATION_TYPE",
"ORGANIZATION_TYPE",
"REG_CITY_NOT_LIVE_CITY",
"REG_CITY_NOT_WORK_CITY",
"REG_REGION_NOT_LIVE_REGION",
"REG_REGION_NOT_WORK_REGION",
"REGION_POPULATION_RELATIVE",
"REGION_RATING_CLIENT",
"WEEKDAY_APPR_PROCESS_START",
"ID"], axis =1)

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from pandas.api.types import is_numeric_dtype

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

for column in data.columns:
    if not is_numeric_dtype(data[column]):
      data[column] = encoder.fit_transform(data[[column]])

import importlib.util, os
spec = importlib.util.spec_from_file_location("fill", "/content/drive/My Drive/Statistella/data_compiler.py")
fill = importlib.util.module_from_spec(spec)
spec.loader.exec_module(fill)

fill.compile(data, "mean");

print(data.head(20))
from sklearn.model_selection import train_test_split
Y = data['TARGET']
X = data.drop(['TARGET', "DAYS_BIRTH"], axis = 1)
X[["DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "OWN_CAR_AGE"]] /= 500
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(Y_train)

        ID  TARGET  NAME_CONTRACT_TYPE  CODE_GENDER  FLAG_OWN_CAR  \
0   196348       0                 1.0          1.0           1.0   
1   147976       0                 1.0          0.0           1.0   
2    52662       0                 0.0          1.0           1.0   
3   101577       0                 1.0          1.0           1.0   
4   173078       0                 1.0          1.0           0.0   
5    10383       0                 1.0          1.0           1.0   
6    59786       0                 1.0          1.0           1.0   
7   129341       0                 1.0          1.0           1.0   
8   147812       0                 1.0          1.0           1.0   
9    87244       1                 1.0          0.0           0.0   
10   69028       0                 1.0          0.0           0.0   
11   76892       0                 1.0          0.0           0.0   
12  212886       0                 1.0          1.0           1.0   
13   15888       0                

In [ ]:

model1 = Sequential([
    Dense(units = 8, activation = 'relu', bias_regularizer = regularizers.L1(0.01)),
    Dense(units = 4, activation = 'relu'),
    Dense(units = 1, activation = 'sigmoid'),
])
model1.compile(optimizer = Adam(learning_rate = 1e-5), loss = BinaryCrossentropy())
model1.fit(X_train, Y_train, epochs = 30)


In [96]:
model2 = Sequential([
    Dense(units = 3, activation = 'linear', bias_regularizer = regularizers.L2(0.001)),
    #Dense(units = 2, activation = 'relu'),
    Dense(units = 2, activation = 'softmax'),
])

model2.compile(optimizer = Adam(learning_rate = 1e-5), loss = SparseCategoricalCrossentropy())
model2.fit(X_train, Y_train, epochs = 5)

Epoch 1/5
5382/5382 [==============================] - 8s 1ms/step - loss: 6017.1274
Epoch 2/5
5382/5382 [==============================] - 8s 2ms/step - loss: 2903.6467
Epoch 3/5
5382/5382 [==============================] - 8s 2ms/step - loss: 1087.5634
Epoch 4/5
5382/5382 [==============================] - 8s 1ms/step - loss: 720.0831
Epoch 5/5
5382/5382 [==============================] - 8s 1ms/step - loss: 581.1011


In [81]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2')
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(Y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.92      1.00      0.96     39629
           1       0.00      0.00      0.00      3423

    accuracy                           0.92     43052
   macro avg       0.46      0.50      0.48     43052
weighted avg       0.85      0.92      0.88     43052



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [75]:
model1.save('/content/drive/My Drive/Statistella/model1.h5')
# Y_pred = model1.predict(X_test)
# submission = []



# # print(Y_pred)

# from sklearn.metrics import roc_curve

# # Compute ROC curve
# fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)

# # Optimal threshold (You can choose based on your criterion)
# optimal_idx = (tpr - fpr).argmax()
# optimal_threshold = thresholds[optimal_idx]
# print("Optimal Threshold:", optimal_threshold)

NameError: name 'model1' is not defined

In [89]:
Y_pred = model2.predict(X_test)
submission1 = []
for i in Y_pred:
  if i[0] < i[1]:
    submission1.append(0)
  else:
    submission1.append(1)
submissions = np.array(submission1)


print(classification_report(Y_test, submission1))

submission2 = []
for i in Y_pred:
  if i[0] > i[1]:
    submission2.append(0)
  else:
    submission2.append(1)
submissions = np.array(submission2)


print(classification_report(Y_test, submission2))

#Best 0.22, 0.15 model2_A submission1
# 0.93 0.12 model2_B submission2


1346/1346 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

           0       0.88      0.00      0.00     39629
           1       0.08      1.00      0.15      3423

    accuracy                           0.08     43052
   macro avg       0.48      0.50      0.07     43052
weighted avg       0.82      0.08      0.01     43052

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     39629
           1       0.12      0.00      0.00      3423

    accuracy                           0.92     43052
   macro avg       0.52      0.50      0.48     43052
weighted avg       0.86      0.92      0.88     43052



In [59]:

model2.save('/content/drive/My Drive/Statistella/model2_B.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [97]:
# from tensorflow.keras.models import load_model

# # Load the model
# model = load_model('/content/drive/My Drive/Statistella/model2_B.h5')
# model.summary()
!$pip install pytables
import pandas as pd

# Load the .h5 file
df = pd.read_hdf('/content/drive/My Drive/Statistella/model2_B.h5')
print(df.head())

SyntaxError: invalid syntax (<ipython-input-97-34a7eadef088>, line 6)

In [ ]:
drive.flush_and_unmount()